In [1]:
import boto3
import pandas as pd
import numpy as np

In [2]:
def download_file_from_s3(bucket, object_name, local_file_name):
    """
    Download a file from S3 to the local file system.

    :param bucket: Name of the S3 bucket
    :param object_name: S3 object name
    :param local_file_name: Local file name to save the downloaded file
    """
    # # When using IAM roles, boto3 retrieves credentials from the instance metadata
    # s3_client = boto3.client('s3')

    #When setting up credentials locally, use the following code
    session = boto3.Session()
    s3_client = session.client('s3')

    s3_client.download_file(bucket, object_name, local_file_name)

bucket_name = 'myukdata'
s3_file_name = 'Original/Member/Member_GovernmentPosts.csv'
local_file = 'Member_GovernmentPosts.csv'

download_file_from_s3(bucket_name, s3_file_name, local_file)

In [3]:
df = pd.read_csv('Member_GovernmentPosts.csv')
df.info()
pd.set_option('display.max_rows', None)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3626 entries, 0 to 3625
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   memberID            3626 non-null   int64  
 1   house               3579 non-null   float64
 2   name                3626 non-null   object 
 3   id                  3626 non-null   int64  
 4   startDate           3626 non-null   object 
 5   endDate             3483 non-null   object 
 6   additionalInfo      3617 non-null   object 
 7   additionalInfoLink  2705 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 226.8+ KB


,memberID,house,name,id,startDate,endDate,additionalInfo,additionalInfoLink
0,3,1.0,Assistant Whip (HM Treasury),351,2003-06-13T00:00:00,2005-05-10T00:00:00,HM Treasury,https://www.gov.uk/government/organisations/hm...
1,5,1.0,Attorney General (Law Officers),433,1992-04-10T00:00:00,1997-05-01T00:00:00,Law Officers,http://www.attorneygeneral.gov.uk/
2,5,1.0,Solicitor General (Law Officers),434,1987-06-15T00:00:00,1992-04-10T00:00:00,Law Officers,http://www.attorneygeneral.gov.uk/
3,5,1.0,Parliamentary Under-Secretary (Department of H...,406,1986-09-10T00:00:00,1987-06-13T00:00:00,Department of Health and Social Security,NaN
4,7,1.0,Deputy Chief Whip and Treasurer,48,1996-07-23T00:00:00,1997-05-01T00:00:00,HM Household,NaN


In [4]:
df.drop(columns=['additionalInfo',
                 'additionalInfoLink',
                 'house',
                 'id'], inplace=True)

df.rename(columns={'memberID': 'memberId',
                     'name': 'postName'}, inplace=True)
df['startDate'] = pd.to_datetime(df['startDate'])
df['endDate'] = pd.to_datetime(df['endDate'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3626 entries, 0 to 3625
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   memberId   3626 non-null   int64         
 1   postName   3626 non-null   object        
 2   startDate  3626 non-null   datetime64[ns]
 3   endDate    3483 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 113.4+ KB


In [6]:
#Save the file to S3
import boto3
import io

def upload_df_to_s3(df, bucket, object_name):
    """
    Upload a DataFrame to an S3 bucket as CSV.

    :param df: DataFrame to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name
    :return: True if the DataFrame was uploaded, else False
    """
    # Create a buffer
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)

    # Move to the start of the buffer
    csv_buffer.seek(0)

    # Upload the buffer content to S3
    s3_client = boto3.client('s3')
    try:
        s3_client.put_object(Bucket=bucket, Key=object_name, Body=csv_buffer.getvalue())
    except ClientError as e:
        logging.error(e)
        return False
    return True

bucket_name = 'myukdata'
object_name = 'Preprocessing/Member/MemberGovernmentPosts.csv'
upload_df_to_s3(df, bucket_name, object_name)


True